<a href="https://colab.research.google.com/github/Nefarian1/STQD6024-Machine-Learning/blob/main/STQD6024_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pan Zhangyu P136922 STQD6024 Assignment 1🤓🤓🤓
+ link of data: https://archive.ics.uci.edu/dataset/186/wine+quality
+ Two datasets are included, related to red and white vinho verde wine samples, from the north of Portugal. The goal is to model wine quality based on physicochemical tests (see [Cortez et al., 2009], http://www3.dsi.uminho.pt/pcortez/wine/).
+ The two datasets are related to red and white variants of the Portuguese "Vinho Verde" wine. For more details, consult: http://www.vinhoverde.pt/en/ or the reference [Cortez et al., 2009].  Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

#I choosse winequality-red.csv

I also saved the file myself extra, maybe you can view it here：

https://github.com/Nefarian1/STQD6024-Machine-Learning.git

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
file_path = "/content/winequality-red.csv"#I use colab, otherwise it is 'data/winequality-red.csv'
wine_df = pd.read_csv(file_path, sep=";")

In [ ]:
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


#Forward Selection with k-fold Cross Validation

In [ ]:
import pandas as pd
import numpy as np
import itertools
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
# 定义前向选择函数 Define forward selection function
def forward_selection(X, y, k):
    best_features = []
    best_model = None
    best_cv_error = float('inf')

    # 创建 k 折交叉验证 Create k-fold cross validation
    kf = KFold(n_splits=k)

    # 外部循环遍历所有特征数量 The outer loop iterates through all feature quantities
    for i in range(1, len(X.columns) + 1):
        cv_errors = []
        predictors = []

        # 内部循环遍历特征子集 Inner loop iterates over feature subsets
        for combo in itertools.combinations(X.columns, i):
            combo = list(combo)

            # 执行交叉验证 Perform cross-validation
            for train_index, test_index in kf.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                # 拟合模型 Fit model
                model = sm.OLS(y_train, X_train[combo])
                result = model.fit()

                # 计算交叉验证误差 Calculate cross-validation error
                cv_error = np.mean((result.predict(X_test[combo]) - y_test) ** 2)
                cv_errors.append(cv_error)

                # 保存最佳模型和特征子集 Save the best model and feature subset
                if cv_error < best_cv_error:
                    best_cv_error = cv_error
                    best_features = combo
                    best_model = result

        # 打印每个特征数量下的交叉验证误差 Print the cross-validation error for each number of features
        print(f"Features: {best_features}, CV Error: {best_cv_error}")

    return best_features, best_model


In [ ]:
#  提取自变量和因变量 Extract independent and dependent variables
X = wine_df.drop('quality', axis=1)
y = wine_df['quality']

# 执行前向选择与 k 折交叉验证 Perform forward selection and k-fold cross-validation
best_features, best_model = forward_selection(X, y, k=10)

# 打印最佳特征子集和模型摘要 Print the best feature subset and model summary
print("Best features:", best_features)
print("Best model summary:")
print(best_model.summary())

Features: ['alcohol'], CV Error: 0.4838306965339389
Features: ['volatile acidity', 'density'], CV Error: 0.4130776712637133
Features: ['volatile acidity', 'pH', 'sulphates'], CV Error: 0.3560383385726631
Features: ['fixed acidity', 'volatile acidity', 'pH', 'sulphates'], CV Error: 0.344996786954477
Features: ['fixed acidity', 'volatile acidity', 'chlorides', 'pH', 'sulphates'], CV Error: 0.3399464725874838
Features: ['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorides', 'pH', 'sulphates'], CV Error: 0.33964530747256083
Features: ['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorides', 'pH', 'sulphates'], CV Error: 0.33964530747256083
Features: ['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorides', 'pH', 'sulphates'], CV Error: 0.33964530747256083
Features: ['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorides', 'pH', 'sulphates'], CV Error: 0.33964530747256083
Features: ['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorid

This model has a high R-squared value, which means that the model can explain the variability of the target variable well. Furthermore, the p-values ​​for each coefficient of the model are very small, indicating that they are all significant.

The final optimal feature subset is ['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorides', 'pH', 'sulphates'], and the corresponding cross-validation error is 0.3396.

该模型具有较高的R平方值，这意味着该模型可以很好地解释目标变量的变异性。此外，模型每个系数的 p 值都非常小，表明它们都是显着的。

最终最优特征子集为[‘固定酸度’、‘挥发酸度’、‘残留糖’、‘氯化物’、‘pH’、‘硫酸盐’]，对应的交叉验证误差为0.3396。

#Backward with k-fold cross-validation

In [ ]:
# 定义后向选择函数 Define backward selection function
def backward_selection(X, y, k):
    features = list(X.columns)
    best_features = features.copy()
    best_model = None
    best_cv_error = float('inf')

    # 创建 k 折交叉验证 Create k-fold cross validation
    kf = KFold(n_splits=k)

    # 外部循环遍历特征数量 The outer loop iterates over the number of features
    for i in range(len(X.columns), 0, -1):
        cv_errors = []

        # 内部循环遍历特征子集 Inner loop iterates over feature subsets
        for combo in itertools.combinations(features, i):
            combo = list(combo)

            # 执行交叉验证 Perform cross-validation
            for train_index, test_index in kf.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                # 拟合模型 Fit model
                model = sm.OLS(y_train, X_train[combo])
                result = model.fit()

                # 计算交叉验证误差 Calculate cross-validation error
                cv_error = np.mean((result.predict(X_test[combo]) - y_test) ** 2)
                cv_errors.append(cv_error)

                # 保存最佳模型和特征子集 Save the best model and feature subset
                if cv_error < best_cv_error:
                    best_cv_error = cv_error
                    best_features = combo
                    best_model = result

        # 打印每个特征数量下的交叉验证误差 Print the cross-validation error for each number of features
        print(f"Features: {best_features}, CV Error: {best_cv_error}")

        # 移除最不重要的特征 Remove least important features
        if len(features) > 1 and best_features[-1] in features:
            features.remove(best_features[-1])

    return best_features, best_model

# 提取自变量和因变量 Extract independent and dependent variables
X = wine_df.drop('quality', axis=1)
y = wine_df['quality']

# 执行后向选择与 k 折交叉验证 Perform backward selection and k-fold cross-validation
best_features, best_model = backward_selection(X, y, k=10)

# 打印最佳特征子集和模型摘要 Print the best feature subset and model summary
print("Best features:", best_features)
print("Best model summary:")
print(best_model.summary())

Features: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'], CV Error: 0.36259034194079487
Features: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates'], CV Error: 0.3572137158772962
Features: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates'], CV Error: 0.3572137158772962
Features: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates'], CV Error: 0.3572137158772962
Features: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates'], CV Error: 0.

The best feature subset includes ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH' , 'sulphates'], the corresponding cross-validation error is 0.357.

The cross-validation error for the forward-selected model is 0.3396, while the cross-validation error for the backward-selected model is 0.3572. Therefore, the forward-selected model slightly outperforms the backward-selected model in terms of cross-validation error.

最佳特征子集包括[‘固定酸度’、‘挥发性酸度’、‘柠檬酸’、‘残留糖’、‘氯化物’、‘游离二氧化硫’、‘总二氧化硫’、‘密度’、‘pH’、 '硫酸盐']，相应的交叉验证误差为0.357。

前向选择模型的交叉验证误差为 0.3396，而后向选择模型的交叉验证误差为 0.3572。因此，前向选择模型在交叉验证误差方面略优于后向选择模型。

#Forward-backward regressions with k-fold cross-validation

This is not important. It is my own note and will not be translated.

将数据集划分为k个折叠（folds）。

对于每个折叠，执行前向选择和后向选择，使用剩余的k-1个折叠进行交叉验证。在前向选择中，每次添加一个特征，然后进行交叉验证。在后向选择中，每次删除一个特征，然后进行交叉验证。

计算每个特征子集的交叉验证误差，并记录最佳的特征子集和对应的模型。

重复步骤2和3，直到所有的折叠都被用于交叉验证。

最终，选择具有最低交叉验证误差的特征子集作为最佳模型。

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import statsmodels.api as sm

def forward_selection(X, y, k):
    best_features = []
    best_model = None
    best_cv_error = float('inf')

    for i in range(X.shape[1]):
        remaining_features = [f for f in X.columns if f not in best_features]
        models_cv = []

        for feature in remaining_features:
            features_subset = best_features + [feature]
            cv_errors = k_fold_cross_validation(X[features_subset], y, k)
            avg_cv_error = np.mean(cv_errors)
            models_cv.append((features_subset, avg_cv_error))

        best_features_subset, best_cv_error_subset = min(models_cv, key=lambda x: x[1])

        if best_cv_error_subset < best_cv_error:
            best_features = best_features_subset
            best_cv_error = best_cv_error_subset
            X_subset = X[best_features]
            X_subset = sm.add_constant(X_subset)
            best_model = sm.OLS(y, X_subset).fit()
            best_model.cv_error = best_cv_error

    return best_features, best_model


def backward_selection(X, y, k):
    features = list(X.columns)
    best_features = features
    best_model = None
    best_cv_error = float('inf')

    for i in range(len(features), 0, -1):
        models_cv = []

        for feature in features:
            features_subset = [f for f in features if f != feature]
            cv_errors = k_fold_cross_validation(X[features_subset], y, k)
            avg_cv_error = np.mean(cv_errors)
            models_cv.append((features_subset, avg_cv_error))

        best_features_subset, best_cv_error_subset = min(models_cv, key=lambda x: x[1])

        if best_cv_error_subset < best_cv_error:
            best_features = best_features_subset
            best_cv_error = best_cv_error_subset
            X_subset = X[best_features]
            X_subset = sm.add_constant(X_subset)
            best_model = sm.OLS(y, X_subset).fit()
            best_model.cv_error = best_cv_error
            features = best_features_subset

    return best_features, best_model


def k_fold_cross_validation(X, y, k):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    cv_errors = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = sm.OLS(y_train, X_train).fit()
        y_pred = model.predict(X_test)
        cv_error = ((y_pred - y_test) ** 2).mean()
        cv_errors.append(cv_error)

    return cv_errors


# 使用前向选择进行回归 Regression using forward selection
best_features_forward, best_model_forward = forward_selection(wine_df.drop('quality', axis=1), wine_df['quality'], k=10)
print("Forward selection best features:", best_features_forward)
print("Forward selection best model summary:")
print(best_model_forward.summary())
print("CV Error:", best_model_forward.cv_error)

# 使用后向选择进行回归 Regression using backward selection
best_features_backward, best_model_backward = backward_selection(wine_df.drop('quality', axis=1), wine_df['quality'], k=10)
print("\nBackward selection best features:", best_features_backward)
print("Backward selection best model summary:")
print(best_model_backward.summary())
print("CV Error:", best_model_backward.cv_error)


Forward selection best features: ['alcohol', 'sulphates', 'fixed acidity', 'volatile acidity', 'density', 'chlorides', 'total sulfur dioxide', 'pH', 'free sulfur dioxide']
Forward selection best model summary:
                            OLS Regression Results                            
Dep. Variable:                quality   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     99.10
Date:                Wed, 08 May 2024   Prob (F-statistic):          6.48e-147
Time:                        06:17:36   Log-Likelihood:                -1570.4
No. Observations:                1599   AIC:                             3161.
Df Residuals:                    1589   BIC:                             3215.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         


Forward selection model:
+ Adjusted R-squared value: 0.356
+ AIC: 3161.0
+ CV Error: 0.42473854266162026

Backward selection model:
+ Adjusted R-squared value: 0.356
+ AIC: 3159.0
+ CV Error: 0.4237155091395025

The adjusted R-squared values ​​of the two models are very close, both around 0.356, which means that they perform similarly in explaining the target variable. However, according to AIC, the backward selection model has a slightly lower value of 3159.0, which is smaller than the forward selection model of 3161.0. The cross-validation error is lower (0.4237155091395025), and lower AIC values ​​are better, indicating that the backward selection model is slightly better at fitting the data.

这两个模型的调整后 R-squared 值非常接近，都在 0.356 左右，这意味着它们在解释目标变量方面的性能相似。但是，根据 AIC，后向选择模型的值稍微低一些，为 3159.0，比前向选择模型的 3161.0 小。交叉验证误差更低（0.4237155091395025），AIC值越低越好，这表明后向选择模型在拟合数据时稍微好一些。

#Summarize

🤓🤓🤓

🤓Forward Selection with k-fold Cross Validation:
+ Best features: ['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorides', 'pH', 'sulphates']
+ CV Error: 0.33964530747256083
+ R-squared: 0.983
+ Adjusted R-squared: 0.983
+ AIC: 3245
+ BIC: 3277

🤓Backward Selection with k-fold Cross Validation:
+ Best features: ['volatile acidity', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
+ CV Error: 0.3572137158772962
+ R-squared: 0.984
+ Adjusted R-squared: 0.984
+ AIC: 3136
+ BIC: 3189

🤓Forward-Backward Regression with k-fold Cross Validation:
+ Forward selection of the best features: ['alcohol', 'sulphates', 'fixed acidity', 'volatile acidity', 'density', 'chlorides', 'total sulfur dioxide', 'pH', 'free sulfur dioxide']
+ Backward selection of the best features: ['volatile acidity', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
+ CV Error: （0.4237155091395025<0.42473854266162026）
+ R-squared: 0.360
+ Adjusted R-squared: 0.356
+ AIC: (3159<3161)
+ BIC: (3207<3215)


🤓🤓🤓

The Forward Selection model has the lowest CV Error, and the Backward Selection model has the smallest AIC and BIC values. I prefer the Forward Selection model.